In [1]:
#Install necessary libraries

!pip install --upgrade pip
!pip install networkx==2.1
!pip install geopy
!pip install srtm.py
!pip install geohelper

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (18.1)


In [0]:
#Import the necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import geopy
from geopy.distance import vincenty
from geopy.distance import VincentyDistance
import srtm
from geohelper import distance 
from geohelper import bearing
from datetime import datetime
import csv
import os

import networkx as nx
from networkx import *


In [3]:
# Download data

! pip install googledrivedownloader

In [0]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1pz4Cw2bowp71pKIgFF86kUNHR1Rk1CZx',
                                    dest_path='./CSV_files.zip',
                                    unzip=False)

In [5]:
! unzip -q CSV_files.zip

replace 20181106_data_upm.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [6]:
#Import file

directory = os.getcwd() + '/'
csvfile = '20181106_data_upm_los.csv'

df_towers = pd.read_csv(directory+csvfile, sep='|')
df_towers

,tower_id_1,latitude_1,longitude_1,tower_height_1,tower_id_2,latitude_2,longitude_2,tower_height_2,distance
0,314,-5.905003,-76.101394,15.0,6989,-5.901750,-76.101667,70.0,361.001219
1,314,-5.905003,-76.101394,15.0,6983,-5.907081,-76.110300,11.0,1012.603424
2,314,-5.905003,-76.101394,15.0,6997,-5.895262,-76.104769,11.0,1140.206389
3,314,-5.905003,-76.101394,15.0,6990,-5.900704,-76.111505,11.0,1216.375621
4,314,-5.905003,-76.101394,15.0,6999,-5.894595,-76.105726,11.0,1246.941812
5,314,-5.905003,-76.101394,15.0,6995,-5.897890,-76.110190,30.0,1251.972450
6,314,-5.905003,-76.101394,15.0,7000,-5.894028,-76.104556,20.0,1263.188748
7,314,-5.905003,-76.101394,15.0,7001,-5.893689,-76.104540,20.0,1298.763767
8,314,-5.905003,-76.101394,15.0,326,-5.894089,-76.106814,30.0,1347.945480
9,314,-5.905003,-76.101394,15.0,320,-5.888932,-76.112955,30.0,2190.321843


In [0]:
#Define function: Line of sight

def check_los(origin, destination, tower_height_1, tower_height_2, interval):
    "This function checks if there is LoS between two points or not"
    
    #Import altitude API
    elevation_data = srtm.get_data()
    
    #Get initial and final altitudes
    H1 = elevation_data.get_elevation(origin.latitude, origin.longitude)
    H2 = elevation_data.get_elevation(destination.latitude, destination.longitude)
    
    #Initialize the error flag that returns True if there has been any issue with the API
    flag_error = False
    
    #Bearing (orientation or azimuth, same thing) needed to go in a straight line from origin to destination
    azimuth = bearing.initial_compass_bearing(origin.latitude, origin.longitude, destination.latitude, destination.longitude)
    
    #Distance origin-destination
    distance_total = distance.get_distance(origin.latitude, origin.longitude, destination.latitude, destination.longitude)
    
    #If D < interval, we assume LoS is True
    if(distance_total <= interval):
        line_of_sight = True
        flag_error = False
        return (line_of_sight, flag_error, pd.DataFrame({'Lat': [], 'Lon': [], 'Alt': []}), 0, 0)
    
    #Number of intermediate points where we will evaluate the altitude
    number_segments = int(distance_total/interval)
    
    #Initialize the current point variable and the list of latitudes, longitudes, altitudes and intermediate distances
    current_point = origin
    lats = []
    longs = []
    alts = []
    dis = []
    
    #Insert the first set of data
    lats.insert(len(lats),float(origin.latitude))
    longs.insert(len(longs),float(origin.longitude))
    alts.insert(len(alts),float(H1))
    dis.insert(len(dis),0)
    
    #Initialize line_of_sight to True and rest of variables to zero
    line_of_sight = True
    max_alt = 0
    
    delta_height_tower_1 = 0
    delta_height_tower_2 = 0
    max_dt1 = 0
    max_dt2 = 0
    
    #We start evaluating the altitude of all intermediate points between origin and destination
    for y in range(0, number_segments):
        
        #Evaluate altitude and distance of intermediate point
        destination_intermediate = VincentyDistance(meters=interval).destination(current_point, azimuth)
        altitude = elevation_data.get_elevation(destination_intermediate.latitude, destination_intermediate.longitude)
        di = distance.get_distance(origin.latitude, origin.longitude, destination_intermediate.latitude, destination_intermediate.longitude)
        
        #If the API returned a valid altitude, we evaluate conditions. Otherwise we skip the process and we will return
        #Line of sight as True and flag_error as True as well
        if (altitude):
            
            #Geometric conditions to be met if there is NOT line of sight because of this pooint
            if(altitude > H1 + tower_height_1 + (H2 - H1 + tower_height_2 - tower_height_1)*di/distance_total):
                
                #In this case there is no line of sight and we proceed to calculate geometrically the additional altitude we would
                #need at both ends (separately) in order to have line of sight
                line_of_sight = False
                delta_height_tower_1 = (di/(distance_total - di))*((altitude - H1)*distance_total/di + H1 - H2 - tower_height_2) - tower_height_1
                delta_height_tower_2 = (altitude - H1 - tower_height_1)*distance_total/di + H1 - H2 + tower_height_1 - tower_height_2
                
                #We update the value of these additional altitudes if it is greater than the additional altitudes calculated previously
                if(delta_height_tower_1 > max_dt1):
                    max_dt1 = delta_height_tower_1
                if(delta_height_tower_2 > max_dt2):
                    max_dt2 = delta_height_tower_2
            
            #Insert at the end of the lists of points the new values obtained
            lats.insert(len(lats),float(destination_intermediate.latitude))
            longs.insert(len(longs),float(destination_intermediate.longitude))
            alts.insert(len(alts),float(altitude))
            dis.insert(len(dis),float(di))
        
        #Update intermediate point for next iteration
        current_point = destination_intermediate
    
    #Once we finish evaluating everything we create a data frame with all the information
    data = [('Lat', lats), ('Long', longs), ('Alt', alts), ('Di', dis)]
    df = pd.DataFrame.from_items(data,columns=['Lat','Long', 'Alt', 'Di'])
    
    #If we do not have enough points (either 0,1 or it failed to get the altitude of 20% of more of the intermediate points)
    if(len(df) <= 1 or len(df)<0.5*number_segments):
        line_of_sight = False
        flag_error = True
    
    #We return line of sight (True or False), flag_error (True or False), data frame with the data, and the max values of the 
    #additional height needed.
    return (line_of_sight, flag_error, df, max_dt1, max_dt2)

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [0]:
#STEP 1: Calculate line of sight between tower 7465 and its sourroundings

#We check the surroundings of tower 7465
tower_fiber = 7465
df_surroundings = ???

#These are the two towers at a distance < 40 km
tower_id_1 = ???
tower_id_2 = ???

#We define the geographical points
origin = geopy.Point(df_towers.ix[df_towers['tower_id_1'] == tower_fiber]['latitude_1'].values[0], df_towers.ix[df_towers['tower_id_1'] == tower_fiber]['longitude_1'].values[0])
destination_1 = geopy.Point(df_towers.ix[df_towers['tower_id_1'] == tower_id_1]['latitude_1'].values[0], df_towers.ix[df_towers['tower_id_1'] == tower_id_1]['longitude_1'].values[0])
destination_2 = geopy.Point(df_towers.ix[df_towers['tower_id_2'] == tower_id_2]['latitude_2'].values[0], df_towers.ix[df_towers['tower_id_2'] == tower_id_2]['longitude_2'].values[0])

#We extract the tower heights
tower_height_fiber = df_towers.ix[df_towers['tower_id_1'] == tower_fiber]['tower_height_1'].values[0]
tower_height_1 = df_towers.ix[df_towers['tower_id_1'] == tower_id_1]['tower_height_1'].values[0]
tower_height_2 = df_towers.ix[df_towers['tower_id_2'] == tower_id_2]['tower_height_2'].values[0]

#We define interval
interval = 40

#We check if there is line of sight
output_1 = ???
output_2 = ???


#Print the outputs that help us determine the quality
print_full(output_1[2])
print_full(output_2[2])

In [0]:
#STEP 2: Line of sight for all towers

#Interval for Line of sight
interval = 40 #meters

#Add some fields
df_towers['line_of_sight'] = False
df_towers['additional_height_tower_1'] = math.nan
df_towers['additional_height_tower_2'] = math.nan
df_towers['error_flag'] = False

#Import API for elevation
elevation_data = srtm.get_data()

#Total iterations
n_iterations = len(df_towers)
total_iterations = 0

for x in range(0, n_iterations):
    if x%100==0:
        print("Iteration: " + str(x) + "/" + str(n_iterations) + "   " + str(datetime.now()))
    
    #Tower IDs
    tower_1 = df_towers['tower_id_1'].iloc[x]
    tower_2 = df_towers['tower_id_2'].iloc[x]
    
    
    #Tower heights
    if(pd.isnull(df_towers['tower_height_1'][x])):
        tower_height_1 = 0
    else:
        tower_height_1 = df_towers['tower_height_1'].iloc[x]
    
    if(pd.isnull(df_towers['tower_height_2'][x])):
        tower_height_2 = 0
    else:
        tower_height_2 = df_towers['tower_height_2'].iloc[x]

    #Origin and destination
    origin = geopy.Point(df_towers['latitude_1'][x], df_towers['longitude_1'][x])
    destination = geopy.Point(df_towers['latitude_2'][x], df_towers['longitude_2'][x])
    
    #We calculate altitudes of both points
    H1 = elevation_data.get_elevation(origin.latitude, origin.longitude, approximate=True)
    H2 = elevation_data.get_elevation(destination.latitude, destination.longitude, approximate=True)
        
    if(pd.isnull(H1) or pd.isnull(H2)):
        df_towers['line_of_sight'][x] = False
        df_towers['error_flag'][x] = True
        print("Skip for lack of H1 H2")
        continue
       
    #If none of the conditions above is true, we evaluate if there is line of sight and save the outputs
    output = ???

    line_of_sight = ???
    flag_los = ???
    df_points = ???
    additional_height_1 = ???
    additional_height_2 = ???
    
    df_towers.set_value(x,'line_of_sight',line_of_sight,False)
    df_towers.set_value(x,'additional_height_tower_1',additional_height_1,False)
    df_towers.set_value(x,'additional_height_tower_2',additional_height_2,False)
    df_towers.set_value(x,'error_flag',flag_los,False)

In [0]:
#Export line-of-sight analysis
output_file_1 = '/output_1'
df_towers.to_csv(directory+'/'+output_file_1, sep='|', index = False)

In [0]:
#Re-load line-of-sight analysis

df_line_of_sight = pd.read_csv(directory+'/'+output_file_1, sep='|')

df_edges = df_line_of_sight.ix[df_line_of_sight['line_of_sight'] == True]

df_edges

In [0]:
#STEP 3: Create graph/graphs

#This is a Graph object defined in NetworkX
G = nx.Graph()

#We now add the edges to the graph (bi-directional) HINT: use add_edge() function
for z in range(0, len(df_edges)):
    ???

#We check if there is just one single connected subgraph or more than one
graphs = list(nx.connected_component_subgraphs(G))
len(graphs)

In [0]:
#Since there are two connected subgraphs, we will work with both of them separately
G1 = graphs[0]
G2 = graphs[1]

#We check which one of the two have fiber PoPs
G1['6995'] #Works fine
#G1['7465'] #Draws error

#G2['6995'] #Draws error
#G2['7465'] #Draws error

In [0]:
#Design 1: connecting Iquitos, Mazán and the Jungle from  Yurimaguas

#From Yurimaguas (tower 6995)
#1. Is there any way I can reach tower 81101 (the farthest one) without building any further tower? How? How many hops?
#2. With this design: How many hops do I need to get to Iquitos (7766)
#3. And to  Mazán? (371)
#4. Is there any way I can reach Huatape? (7252)

fiber_node = '6995'

#1

paths = ???
lengths = ???

farthest_path = paths['81101']
hops_farthest_path = lengths['81101']

#2
path_iquitos_1 = paths['7766']
length_iquitos_1 = lengths['7766']

#3
path_mazan_1 = paths['371']
length_mazan_1 = lengths['371']

#4
#path_huatape_1 = paths['7252']  #Draws error
farthest_path


In [0]:
#STEP 4: Design 2: connecting the cities of Iquitos and Mazán with budget constraints

#From Requena (tower 7465)
#1. Can I get to Iquitos (7766) with 10 hops?
#2. And to  Mazán? (371)

#We add the edges to the second subgraph


fiber_node_2 = '7465'

paths = ???
lengths = ???

#1
path_iquitos_2 = paths['7766']
length_iquitos_2 = lengths['7766']

#2
path_mazan_2 = paths['371']
length_mazan_2 = lengths['371']
 
path_mazan_2

In [0]:
#Function to calculate the Steiner tree for a graph G and a set of vertices V

from itertools import combinations, chain

from networkx import utils
from networkx.utils import pairwise, not_implemented_for

__all__ = ['metric_closure', 'steiner_tree']

@not_implemented_for('directed')
def metric_closure(G, weight='weight'):
    """  Return the metric closure of a graph.

    The metric closure of a graph *G* is the complete graph in which each edge
    is weighted by the shortest path distance between the nodes in *G* .

    Parameters
    ----------
    G : NetworkX graph

    Returns
    -------
    NetworkX graph
        Metric closure of the graph `G`.

    """
    M = nx.Graph()

    seen = set()
    Gnodes = set(G)
    for u, (distance, path) in nx.all_pairs_dijkstra(G, weight=weight):
        seen.add(u)
        for v in Gnodes - seen:
            M.add_edge(u, v, distance=distance[v], path=path[v])

    return M



@not_implemented_for('directed')
def steiner_tree(G, terminal_nodes, weight='weight'):
    """ Return an approximation to the minimum Steiner tree of a graph.

    Parameters
    ----------
    G : NetworkX graph

    terminal_nodes : list
         A list of terminal nodes for which minimum steiner tree is
         to be found.

    Returns
    -------
    NetworkX graph
        Approximation to the minimum steiner tree of `G` induced by
        `terminal_nodes` .

    Notes
    -----
    Steiner tree can be approximated by computing the minimum spanning
    tree of the subgraph of the metric closure of the graph induced by the
    terminal nodes, where the metric closure of *G* is the complete graph in
    which each edge is weighted by the shortest path distance between the
    nodes in *G* .
    This algorithm produces a tree whose weight is within a (2 - (2 / t))
    factor of the weight of the optimal Steiner tree where *t* is number of
    terminal nodes.

    """
    # M is the subgraph of the metric closure induced by the terminal nodes of
    # G.
    M = metric_closure(G, weight=weight)
    # Use the 'distance' attribute of each edge provided by the metric closure
    # graph.
    H = M.subgraph(terminal_nodes)
    mst_edges = nx.minimum_spanning_edges(H, weight='distance', data=True)
    # Create an iterator over each edge in each shortest path; repeats are okay
    edges = chain.from_iterable(pairwise(d['path']) for u, v, d in mst_edges)
    T = G.edge_subgraph(edges)
    return T

In [0]:
#STEP 5: Design 3: connecting a set of mandatory cities

terminal_nodes = ['7465', '7766', '371', '7904', '76111']

#Use the steiner tree function
st = ???

df_st = pd.DataFrame.from_records(list(st.edges()))


df_st